In [1]:
import requests
import json
import mysql.connector

In [2]:
api_key = "78cf85e6-5f8d-4d1f-aaf4-5c0e84d5a08a"
obj_url = "https://api.harvardartmuseums.org/object"


In [3]:
paintings_data = []

for page in range(25):
    params = {
        "apikey": api_key,
        "classification": "Paintings",
        "size": 100,
        "page": page
    }
    response = requests.get(obj_url, params=params)
    if response.status_code == 200:
        data = response.json()
        paintings_data.extend(data.get("records", []))
        print(f"Page {page + 1}: {len(data.get('records', []))} records")
    else:
        print(f"Failed on page {page + 1}")
        break

print(f"Total Paintings: {len(paintings_data)}")

with open("paintings_raw.json", "w") as f:
    json.dump(paintings_data, f)
print(" Data saved to paintings_raw.json")


paintings_metadata = []
paintings_media = []
paintings_colors = []

for item in paintings_data:
    paintings_metadata.append({
        "id": item.get("id"),
        "title": item.get("title"),
        "culture": item.get("culture"),
        "period": item.get("period"),
        "century": item.get("century"),
        "medium": item.get("medium"),
        "dimensions": item.get("dimensions"),
        "description": item.get("description"),
        "department": item.get("department"),
        "classification": item.get("classification"),
        "accessionyear": item.get("accessionyear"),
        "accessionmethod": item.get("accessionmethod"),
    })

    paintings_media.append({
        "objectid": item.get("id"),
        "image_url": item.get("images", [{}])[0].get("baseimageurl") if item.get("images") else None,
        "description": item.get("images", [{}])[0].get("description") if item.get("images") else None,
    })

    for color in item.get("colors", []):
        paintings_colors.append({
            "objectid": item.get("id"),
            "color": color.get("color"),
            "color_hex": color.get("hex"),
        })


Page 1: 100 records
Page 2: 100 records
Page 3: 100 records
Page 4: 100 records
Page 5: 100 records
Page 6: 100 records
Page 7: 100 records
Page 8: 100 records
Page 9: 100 records
Page 10: 100 records
Page 11: 100 records
Page 12: 100 records
Page 13: 100 records
Page 14: 100 records
Page 15: 100 records
Page 16: 100 records
Page 17: 100 records
Page 18: 100 records
Page 19: 100 records
Page 20: 100 records
Page 21: 100 records
Page 22: 100 records
Page 23: 100 records
Page 24: 100 records
Page 25: 100 records
Total Paintings: 2500
 Data saved to paintings_raw.json


In [4]:
conn = mysql.connector.connect(
    host="host.docker.internal",
    user="root",
    password="root",
    database="harvard_ds"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_metadata (
    id INT PRIMARY KEY,
    title TEXT,
    culture TEXT,
    period TEXT,
    century TEXT,
    medium TEXT,
    dimensions TEXT,
    description TEXT,
    department TEXT,
    classification TEXT,
    accessionyear INT,
    accessionmethod TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_media (
    artifact_id INT,
    image_url TEXT,
    description TEXT,
    FOREIGN KEY (artifact_id) REFERENCES artifact_metadata(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_colors (
    artifact_id INT,
    color TEXT,
    color_hex TEXT,
    FOREIGN KEY (artifact_id) REFERENCES artifact_metadata(id)
)
""")


In [5]:

for row in paintings_metadata:
    cursor.execute("""
        INSERT IGNORE INTO artifact_metadata (
            id, title, culture, period, century, medium,
            dimensions, description, department, classification,
            accessionyear, accessionmethod
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row["id"],
        row["title"],
        row["culture"],
        row["period"],
        row["century"],
        row["medium"],
        row["dimensions"],
        row["description"],
        row["department"],
        row["classification"],
        row["accessionyear"],
        row["accessionmethod"]
    ))


In [6]:
for row in paintings_media:
    cursor.execute("""
        INSERT IGNORE INTO artifact_media VALUES (%s, %s, %s)
    """, tuple(row.values()))


for row in paintings_colors:
    cursor.execute("""
        INSERT IGNORE INTO artifact_colors VALUES (%s, %s, %s)
    """, tuple(row.values()))


conn.commit()
cursor.close()
conn.close()
print(" All data inserted into MySQL successfully.")

 All data inserted into MySQL successfully.
